In [ ]:
!apt-get install pyvcf

In [ ]:
!apt-get install -y tabix
!pip install pytabix


In [ ]:

!grep -o -i PASS HG003_GRCh38_1_22_v4.2.1_benchmark.vcf | wc -l
!grep -o -i FAIL HG003_GRCh38_1_22_v4.2.1_benchmark.vcf | wc -l

In [ ]:
import vcf

#if a variat exist in both a VCF file anf the reference VCF, the variant FILTER is set to PASS. Else, it is FAIL
def update_filter_column(vcf_file, reference_vcf, output_vcf):
    vcf_reader = vcf.Reader(filename=vcf_file)
    reference_reader = vcf.Reader(filename=reference_vcf)

    reference_variants = set((record.POS) for record in reference_reader)

    vcf_writer = vcf.Writer(open(output_vcf, 'w'), vcf_reader)

    for record in vcf_reader:
        variant_key = (record.POS)
        if variant_key in reference_variants:
            record.FILTER = "PASS"

        else:
            record.FILTER = "FAIL"

        vcf_writer.write_record(record)





In [ ]:
# Provide the paths to your VCF files
file_1 = "HG003-bcf.vcf"
file_2 = "HG003_gatk.vcf"
reference_vcf_path = "HG003_GRCh38_1_22_v4.2.1_benchmark.vcf"
output_1 = "/content/HG003_bcf_updated.vcf"
output_2 = "/content/HG003_gatk_updated.vcf"

update_filter_column(file_1, reference_vcf_path, output_2)
update_filter_column(file_2, reference_vcf_path, output_4)


In [ ]:
!apt-get install vcftools


In [ ]:
#print the number of fail variants
!grep -o -i FAIL HG003_bcf_updated.vcf | wc -l
!grep -o -i FAIL HG003_gatk_updated.vcf | wc -l

#print the number of pass variants
!grep -o -i PASS HG003_bcf_updated.vcf | wc -l
!grep -o -i PASS HG003_gatk_updated.vcf | wc -l








In [ ]:
!ls -lh

In [ ]:
!vcf-concat HG003_bcf_updated.vcf \
  HG003_gatk_updated.vcf > HG003_merged.vcf

In [ ]:
ls -lh

In [ ]:
!cat HG003_merged.vcf | vcf-annotate --fill-type | grep -oP "TYPE=\w+" | sort | uniq -c

In [ ]:
!grep -o -i PASS HG003_merged.vcf | wc -l
!grep -o -i FAIL HG003_merged.vcf | wc -l

In [ ]:
#find unique words to be added later in the vocab list for the TRNASFORMERS model
def find_words_between_substring(file_path):
    found_words = []

    with open(file_path, 'r') as file:
        content = file.read()
        start_index = content.find("ID=")

        while start_index != -1:
            start_index += 3  # Length of "ID="
            end_index = content.find(",", start_index)
            if end_index != -1:
                found_words.append(content[start_index:end_index].strip())
            start_index = content.find("ID=", end_index)

    return found_words




In [ ]:
# Usage example
file_path_1 = "HG003_bcf_updated.vcf"  # Replace with your file path
file_path_2 = "HG003_gatk_updated.vcf"
result_1 = find_words_between_substring(file_path_1)
print(result_1)
result_2 = find_words_between_substring(file_path_2)
print(result_2)




In [ ]:
#remove not needed words
print(result_1.index('chr1'))
print(result_2.index('chr1'))


In [ ]:
result_1 = result_1[:21]
result_2 = result_2[1:22]

In [ ]:
out_1 = map(lambda x:x.lower(), result_1)
out_2 = map(lambda x:x.lower(), result_2)


result_1 = list(out_1)
result_2 = list(out_2)


In [ ]:

print(result_1)
print(result_2)


In [ ]:
!head bert-base-uncased-vocab.txt

In [ ]:
!wc -l bert-base-uncased-vocab.txt

In [ ]:
#add unique words to the bert_base_uncased vocabs
with open("bert-base-uncased-vocab.txt", "a") as f:
   for word in result_1:
     f.write(word+'\n')
   for z in result_2:
     f.write(z+'\n')




In [ ]:
!wc -l bert-base-uncased-vocab.txt

In [ ]:
def remove_duplicate_words(file_path, output_file_path):
    unique_words = []
    seen_words = set()

    with open(file_path, 'r') as file:
        for line in file:
            words = line.strip().split()
            for word in words:
                if word not in seen_words:
                    unique_words.append(word)
                    seen_words.add(word)

    with open(output_file_path, 'w') as output_file:
        for word in unique_words:
            output_file.write(word + '\n')
# Usage example
input_file_path = "bert-base-uncased-vocab.txt"  # Replace with your input file path
output_file_path = "unique_word.txt"  # Replace with your desired output file path
remove_duplicate_words(input_file_path, output_file_path)


In [ ]:
!wc -l unique_word.txt


In [ ]:
!tail -26 unique_word.txt

In [ ]:

!awk '/^##/{h=$0; next} /^#CHROM/{print h; print; next} {print}' HG003_merged.vcf > hg003_merged.vcf



In [ ]:
!head hg003_merged.vcf

In [ ]:

!sed -i '1d' hg003_merged.vcf


In [ ]:
!head -15 hg003_merged.vcf

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("hg003_merged.vcf", delimiter='\t', header=None, names=['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'HG003'], low_memory=False)


df=df.drop(df.columns[[0, 1, 2, 3, 4]], axis=1)


# Report the number of sentences.
print('Number of training lines: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
print(df.sample(3, random_state=42))


In [ ]:
df['FILTER'].replace(['PASS', 'FAIL'],
                        [0, 1], inplace=True)


# Display 10 random rows from the data.
df.sample(10, random_state=42)

In [ ]:
df.to_csv('hg003_bcf_gatk.csv', index=False)


In [ ]:
!wc -l unique_word.txt
!wc -l bert-base-uncased-vocab.txt

In [ ]:
ls -lh ./drive/MyDrive/Colab\ Notebooks/deepref